In [3]:
# 필요 모듈 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
# URL = 'https://search.shopping.naver.com/search/category/?catId={}&origQuery&pagingIndex={}&productSet=model'.format(50000842,2)

In [4]:
cate_df = pd.read_csv('NaverCategories.csv')
cate_id = cate_df['카테고리번호'].tolist()
URL = 'https://search.shopping.naver.com/search/category/?catId={}&origQuery&pagingIndex={}&productSet=model'.format(50000842,2)

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

In [18]:
driver_path = '/Users/이나라/.wdm/drivers/chromedriver/win32/113.0.5672/chromedriver.exe' # 드라이버 파일의 실제 경로
service = webdriver.chrome.service.Service(driver_path)
driver = webdriver.Chrome(service=service)

In [7]:
driver.get('https://search.shopping.naver.com/search/category/')

In [ ]:
naver_product_df = pd.DataFrame(columns=['product_name','image_list','price','price_list','brand','cate_id'])

In [41]:
URL = 'https://search.shopping.naver.com/search/category/?catId={}&origQuery&pagingIndex={}&productSet=model'.format(50000842,2)
driver.get(URL)

In [43]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
items_list = driver.find_elements(By.CSS_SELECTOR, '.basicList_list_basis__uNBZx .basicList_item__0T9JD')

In [39]:
items_list[0].text

'해외남성 개량한복 셔츠 반팔 한복셔츠 복고풍 트렌디 의류 Mochae\n최저16,060원판매처 3\n패션의류남성의류한복\n등록일 2023.05.찜하기0정보 수정요청\n쇼핑몰별 최저가\n11번가\n16,060\nG마켓\n16,070\n옥션\n18,900'

In [ ]:
names = []
image_list = []
price = []
price_list = []
for item in items_list :
    product_name = item.find_element(By.CLASS_NAME, 'basicList_title__VfX3c').text
    price_one = item.find_element(By.CLASS_NAME, 'price_num__S2p_v').text
    item.find_element(By.CSS_SELECTOR, '.thumbnail_thumb_wrap__RbcYO').click()



In [35]:
cate_df[cate_df['카테고리번호']==50000842]

,카테고리번호,대분류,중분류,소분류,세분류
4354,50000842,패션의류,남성의류,한복,NaN


In [34]:
cat_list

[['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복'],
 ['패션의류', '남성의류', '한복']]

In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json
import time
import pandas as pd

def get_ns_prd_list(cat_id: str, page_num: int) ->list:
    ua = UserAgent(verify_ssl=False) 
    fake_ua = ua.random
    headers = {
        'user-agent' : fake_ua,
        'cache-control' : 'no-cache, no-store, must-revalidate',
        # -- sbth 없으면 정보 안가져옴 --
        'sbth' : '113119acf20fb30e0fe7b38a68f33e8c6d475cdd2388e97971ed219171c6fb1607d305c25c31fff314b29a15ec6d279c',
        # -- referer 없으면 정보 안가져옴 --
        'referer' : 'https://search.shopping.naver.com/search/category',    
    }

    params = (
        ('sort','rel'),
        ('pagingIndex',page_num),
        ('pagingSize','40'),
        ('viewType','list'),
        # -- 여기에 해당하는 catId는 엑셀파일에 있는 catId --
        # -- 해당 예시는 아마도 슬리퍼 -- 
        ('catId',cat_id)
    )


    url = 'https://search.shopping.naver.com/api/search/category'
    res = requests.get(url, headers=headers, params=params)
    results = res.json()
    results = results['shoppingResult']['products']
    prd_list = list()
    # -- 예시로 id와 productName만 가져와봄 -- 
    for result in results:
        prd_id = result['id']
        prd_name = result['productName']
        cat1_id = result['category1Id']
        cat1_name = result['category1Name']
        cat2_id = result['category2Id']
        cat2_name = result['category2Name']
        cat3_id = result['category3Id']
        cat3_name = result['category3Name']
        prd_image_url = result['imageUrl']
        prd_low_price = result['lowPrice']
        prd_list.append([prd_id, prd_name, cat1_id, cat1_name, cat2_id, 
                         cat2_name, cat3_id, cat3_name, prd_image_url, prd_low_price]) 
    
    return prd_list



def get_prd_info(cat_id: str, page_count: int) -> list:
    """
    cat_id -> 크롤링하고자 하는 category id
    page_count -> 크롤링하고자 하는 페이지 수
    """
    prd_list = list()
    for page_num in range(1, page_count+1): # 250 페이지 가져오기 
        prd_list += get_ns_prd_list(cat_id, page_num)
        print('%d page done'%page_num)
        if page_num % 50 == 0:
            time.sleep(5)
            print('-- paused for 5 sec --')

    return prd_list

df = pd.DataFrame(get_prd_info('50000303', 150), 
             columns = ['prd_id', 'prd_name', 'cat1_id', 'cat1_name', 'cat2_id', 
                         'cat2_name', 'cat3_id', 'cat3_name', 'prd_image_url', 'prd_low_price'])

1 page done
2 page done
3 page done
4 page done
5 page done
6 page done
7 page done
8 page done
9 page done
10 page done
11 page done
12 page done
13 page done
14 page done
15 page done
16 page done
17 page done
18 page done
19 page done
20 page done
21 page done
22 page done
23 page done
24 page done
25 page done
26 page done
27 page done
28 page done
29 page done
30 page done
31 page done
32 page done
33 page done
34 page done
35 page done
36 page done
37 page done
38 page done
39 page done
40 page done
41 page done
42 page done
43 page done
44 page done
45 page done
46 page done
47 page done
48 page done
49 page done
50 page done
-- paused for 5 sec --
51 page done
52 page done
53 page done
54 page done
55 page done
56 page done
57 page done
58 page done
59 page done
60 page done
61 page done
62 page done
63 page done
64 page done
65 page done
66 page done
67 page done
68 page done
69 page done
70 page done
71 page done
72 page done
73 page done
74 page done
75 page done
76 page don

In [4]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json
import time
import pandas as pd
from sbth import get_sbth

def get_ns_prd_list(cat_id: str, page_num: int) -> list:
    sbth = get_sbth()
    ua = UserAgent(verify_ssl=False) 
    fake_ua = ua.random
    headers = {
        'user-agent' : fake_ua,
        'cache-control' : 'no-cache, no-store, must-revalidate',
        # -- sbth 없으면 정보 안가져옴 --
        # -- update 필요할 수 있음 => 날짜 정보가 들어가 만료가 될 수 있음 -- 
        'sbth' : sbth,
        # -- referer 없으면 정보 안가져옴 --
        'referer' : 'https://search.shopping.naver.com/search/category', 
    }

    params = (
        ('sort','rel'),
        ('pagingIndex',page_num),
        ('pagingSize','40'),
        ('viewType','list'),
        # -- 여기에 해당하는 catId는 엑셀파일에 있는 catId --
        ('catId',cat_id)
    )


    url = 'https://search.shopping.naver.com/api/search/category'
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        results = res.json()
        results = results['shoppingResult']['products'] # products 키 값안에 상품 정보 들어있음. 
        
        prd_list = list()
        for result in results:
            prd_id = result['id']
            prd_name = result['productName']
            cat1_id = result['category1Id']
            cat1_name = result['category1Name']
            cat2_id = result['category2Id']
            cat2_name = result['category2Name']
            cat3_id = result['category3Id']
            cat3_name = result['category3Name']
            # -- cat 4 name -- 
            
            prd_image_url = result['imageUrl']
            prd_low_price = result['lowPrice']
            prd_list.append([prd_id, prd_name, cat1_id, cat1_name, cat2_id, 
                             cat2_name, cat3_id, cat3_name, prd_image_url, prd_low_price]) 
    else:
        prd_list.append([None, None, None, None, None,
                         None, None, None, None, None]) # 변수들 None 값으로 처리 
        
    return prd_list



def get_prd_info(cat_id: str, page_count = 250):
    """
    cat_id -> 크롤링하고자 하는 category id
    page_count -> 크롤링하고자 하는 페이지 수 (250 pages by default)
    """
    prd_list = list()
    for page_num in range(1, page_count+1): # 250 페이지 가져오기 
        prd_list += get_ns_prd_list(cat_id, page_num)
        print('page %d done'%page_num)
        
        if page_num % 50 == 0:
            print('-- paused for 5 sec in case not to be blocked --')
            time.sleep(5)
            
    print('-- 데이터 개수 :  %d --'%(len(prd_list)))
    
    # -- prd_list에서 데이터 프레임 만들기 --
    df = pd.DataFrame(prd_list, columns = ['prd_id', 'prd_name', 'cat1_id', 'cat1_name', 'cat2_id', 
                                           'cat2_name', 'cat3_id', 'cat3_name', 'prd_image_url', 
                                           'prd_low_price'])
    return df

get_prd_info('50000305', 10)

page 1 done
page 2 done
page 3 done
page 4 done
page 5 done
page 6 done
page 7 done
page 8 done
page 9 done
page 10 done
-- 데이터 개수 :  400 --


,prd_id,prd_name,cat1_id,cat1_name,cat2_id,cat2_name,cat3_id,cat3_name,prd_image_url,prd_low_price
0,86001660206,헤어플러스 원하는대로 골라담아봄 1+1+1 균일가 (샴푸/트리트먼트/에센스),50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_860016...,29900
1,40120096619,바이오틴 샴푸 740ml x 2+ 랩 트리트먼트 200ml,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401200...,29800
2,20621914026,다시마 샴푸 500ml + 다시마 트리트먼트 500ml 세트,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_206219...,57000
3,29650571619,데저티콜라 2종 세트,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_296505...,18000
4,40119892618,세라 오션 브리즈 2종 세트,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401198...,21960
...,...,...,...,...,...,...,...,...,...,...
395,40188334156,케라시스 어드밴스드 리페어 앰플 600ml 샴푸 트리트먼트 세트 손상 컨디셔너 린스,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401883...,14120
396,40188328948,LOMA 데일리 샴푸 바이올렛 컨디셔너 듀오 팩 12온스,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401883...,50000
397,40188340201,GRAHAMS 내추럴 샴푸 250ml 2개+컨디셔너 250ml SET,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401883...,80570
398,40188272889,호유 프로마스터 컬러케어 리치 샴푸 600ml 헤어 트리트먼트 600g 세트,50000002,화장품/미용,50000198,헤어케어,50000305,헤어케어세트,https://shopping-phinf.pstatic.net/main_401882...,71950


In [7]:
a = pd.read_csv('./test.csv',)
a.head(5)

,Unnamed: 0,prd_id,prd_name,cat1_id,cat1_name,cat2_id,cat2_name,cat3_id,cat3_name,prd_image_url,prd_low_price
0,0,85943946803,바이오가 바이오틴 손상모발개선 탈모완화 샴푸(예약판매),50000002,화장품/미용,50000198,헤어케어,50000304,탈모케어,https://shopping-phinf.pstatic.net/main_859439...,27500
1,1,26539564522,블랙 쿠션 15g(SPF34),50000002,화장품/미용,50000194,베이스메이크업,50001370,파운데이션,https://shopping-phinf.pstatic.net/main_265395...,20440
2,2,18900649628,큐어 워터 스플래쉬 쿨링 선스틱 23g(SPF50+),50000002,화장품/미용,50000191,선케어,50000447,선스틱,https://shopping-phinf.pstatic.net/main_189006...,10490
3,3,37951267621,팡팡 빅 선쿠션 시즌7 25g(SPF50+),50000002,화장품/미용,50000191,선케어,50000448,선파우더/쿠션,https://shopping-phinf.pstatic.net/main_379512...,8660
4,4,22356331129,내추럴 퍼펙션 더블 쉴드 선스틱 22g(SPF50+),50000002,화장품/미용,50000191,선케어,50000447,선스틱,https://shopping-phinf.pstatic.net/main_223563...,6620
